# Algoritmo genético 
---
<br/>
Un algoritmo matemático de búsqueda que transforma un conjunto de objetos matemáticos individuales con respecto al tiempo usando operaciones modeladas de acuerdo al principio Darwiniano de reproducción y supervivencia del más apto, y tras haberse presentado de forma natural una serie de operaciones genéticas de entre las que destaca la recombinación sexual

<br/>

## Partes del algoritmo

--- 
<br/>

### Población  <br/>

La población es un conjunto de cromosomas. La población inicial, la cual es generada aleatoriamente está constituida por un conjunto de cromosomas que representan las posibles soluciones del problema. En caso de no hacerlo aleatoriamente, es importante garantizar que dentro de la población inicial, se tenga la diversidad estructural de estas soluciones para tener una representación de la mayor parte de la población posible o al menos evitar la convergencia prematura. La población cambia con cada iteración del algoritmo. 

<br/>

### Función de aptitud <br/>

Esta función es la encargada de otorgar una calificación a cada cromosoma de la población que evalua, la calificación depende de que tan optimo es el cromosoma con respecto a la solución.

<br/>

### Selección <br/>

Después de saber la aptitud de cada cromosoma se seleccionan los cromosomas que seran cruzados, los cromosomas con mayor aptitud serán tendrán mas probabilidad de ser seleccionados. 

<br/>

### Cruzamiento <br/>

El cruzamiento es el principal operador genético, representa la reproducción sexual, opera sobre dos cromosomas a la vez para generar dos descendientes donde se combinan las características de ambos cromosomas padres

<br/>

### Mutación <br/>

Modifica al azar parte del cromosoma de los individuos generados después del cruzamiento, y permite alcanzar zonas del espacio de búsqueda que no estaban cubiertas por los individuos de la población actual. Generalmente solo se cambia una pequeña característica del cromosoma, por ejemplo, cambiar un solo bit aleatorio de 1 a 0

<br/>

### Reemplazo <br/>

Una vez aplicado las operaciones de cruzamiento y mutación, se seleccionan los mejores cromosomas descendientes para que reemplacen a la población

<br/>

In [1]:
import numpy as np
import pygame
import heapq
import random

class Vector:
    """ implementation of Eucledian vector"""

    def __init__(self, x=0.0, y=0.0):
        self.x = x
        self.y = y

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __str__(self):
        return "(" + str(self.x) + " " + str(self.y) + ")"

    # equivalent of Vector(0.0, 0.0)
    def nul(self):
        self.x, self.y = 0.0, 0.0

    # calculates the Eucledian distance between 2 vectors
    def dist(self, other):
        x_dist = self.x - other.x
        y_dist = self.y - other.y
        return np.sqrt(x_dist * x_dist + y_dist * y_dist)

    # transforms the vector to a tuple containing integer values
    # the offset is added to the components of the vector
    def tuple_int(self, offset=0.0):
        return int(self.x + offset), int(self.y + offset)

    # creates a vector with random initial values
    @staticmethod
    def random():
        return Vector(np.random.uniform(0, 2.0) - 1.0, np.random.uniform(0, 2.0) - 1.0)


class Obstacle:
    """ the Obstacle is a rectangle whit which the Rocket can collide
        self.a (type Vector) - contains the point for the upper left corner of the rectangle
        self.b (type Vector) - contains the point for the lower right corner of the rectangle
    """

    def __init__(self, x1, y1, x2, y2):
        self.a = Vector(x1, y1)
        self.b = Vector(x2, y2)

    # return true if the rocket's position is inside the obstacle's rectangle
    def do_collide(self, rocket):
        return self.a.x < rocket.location.x and self.a.y < rocket.location.y and \
                self.b.x > rocket.location.x and self.b.y > rocket.location.y

    # transforms the vector to a tuple containing integer values
    # the offset is added to the components of the vector
    def tuple_int(self, offset):
        return self.a.tuple_int(offset), self.b.tuple_int(offset)


class Rocket:

    def __init__(self, length):

        self.location = Vector(600,350)
        self.acceleration = Vector()
        self.velocity = Vector()

        self.forces = []
        self.length = length

        # this flag is set to False after the rocket did collide with an obstacle
        self.is_alive = True

        for i in range(0, length):
            self.forces.append(Vector.random())

    # calculates the fitness of a Rocket
    # it is recommended to be called after every force vector was added to the rocket
    def fitness(self, target):

        # the fitness value is basically the distance of the rocket from the target point
        # because we want the have a smaller fitness value for larger distances,
        # the inverse value of the distance is used
        inv_dist_to_target = 1.0 / self.location.dist(target)

        # if a collision was detected with an obstacle, penalize the fitness value
        fitness_rate = 1.0
        if not self.is_alive:
            fitness_rate = 0.0000000000000000001

        return inv_dist_to_target * fitness_rate

    # does the recombination between to elements of the population
    def crossover(self, other):

        # a new child is created
        child = Rocket(self.length)

        # generate a random midpoint
        midpoint = np.random.random_integers(1, other.length)

        for i in range(0, other.length):

            # do the recombination by taking force vectors from both elements
            if i < midpoint:
                child.forces[i] = self.forces[i]
            else:
                child.forces[i] = other.forces[i]

        return child

    # mutates the current force vector according to the current mutation rate
    def mutate(self, mutation_rate):
        for i in range(0, self.length):
            rand_value = np.random.rand()
            if rand_value < mutation_rate:
                self.forces[i] = Vector.random()

    # applies a force vector to the rocket's acceleration
    def apply_force(self, force):
        self.acceleration += force

    # applies a force vector to the rocket's acceleration
    # the force value is taken from the self.forces array
    def apply_force_at(self, at):
        self.acceleration += self.forces[at]

    # updates the location of the rocket
    def update(self):

        # update the velocity of the rocket
        self.velocity += self.acceleration

        # update the location of the rocket
        self.location += self.velocity

        # the acceleration of the rocket is set to 0.0
        self.acceleration.nul()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)
bright_red = (255,128,0)
bright_green = (0,255,128)

FPS = 250


class Genetic:

    def __init__(self, title, width, height, iteratons , loc_x, loc_y, population_size=100, mutation_rate=0.1, obstacles=[]):
        # initialize all variables
        self.title = title
        self.width = width
        self.height = height
        self.iteratons = iteratons
        self.target_location = Vector(loc_x, loc_y)
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.population = []
        self.best_child = Rocket(FPS)
        self.obstacles = obstacles

        for i in range(0, self.population_size):
            self.population.append(Rocket(FPS))

    def _next_gen(self):

        fitness_list = []
        new_generation = []

        for member in self.population:

            fitness = (member.fitness(self.target_location)) * 1000
            fitness_list.append((fitness,member))

        new_list = sorted(fitness_list, key=lambda rkt: rkt[0])
        selection_array = []
        for i in range(len(new_list)):
            for j in range(i+1):
                selection_array.append(new_list[i][1])
        
        child_quantity = round(self.population_size * 0.1)
        child_array = []
        for i in range(child_quantity):
            r1 = random.randint(0, len(selection_array)-1)
            r2 = random.randint(0, len(selection_array)-1)
            child = selection_array[r1].crossover(selection_array[r2])
            child.mutate(self.mutation_rate)
            child_array.append(child)

        for i in range(len(new_list)):
            if ( i < child_quantity ):
                new_generation.append(child_array[i])
            else:
                temp = Rocket(FPS)
                temp.forces = new_list[i][1].forces

                new_generation.append(temp)
        self.population = []
        self.population = new_generation

        self.best_child = new_list[len(new_list)-1][1]


    def simulate_with_graphics(self):

        pygame.init()
        game_display = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption(self.title)

        clock = pygame.time.Clock()

        game_exit = False
        counter = 0
        iter_cnt = 0

        while not game_exit and iter_cnt < self.iteratons:

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    game_exit = True

            game_display.fill(WHITE)

            if counter == FPS:
                counter = 0
                iter_cnt += 1
                self._next_gen()

            for member in self.population:

                if member.is_alive:

                    member.apply_force_at(counter)

                    member.update()

                    for obs in self.obstacles:
                        if obs[0] <= member.location.x <= obs[0]+obs[2] and obs[1] <= member.location.y <= obs[1]+obs[3]:
                            member.is_alive = False

                    if member.location.x <= 10 or member.location.x >= 800 or member.location.y <= 10 or member.location.y >= 600 :
                            member.is_alive = False

                pygame.draw.circle(game_display, green, member.location.tuple_int(),10 )

            counter += 1

            pygame.draw.circle(game_display, RED, self.target_location.tuple_int(), 25)
            font = pygame.font.SysFont("monospace", 20)
            #label = font.render("Target", 1, (0, 0, 0))
            #game_display.blit(label, (67, 125))

            for obs in self.obstacles:
                pygame.draw.rect(game_display, blue, obs)

            label = font.render("Generation: " + str(iter_cnt+1), 1, (0, 0, 0))
            game_display.blit(label, (10, 512))
            label = font.render("Best Fitness: " + str(self.best_child.fitness(self.target_location)), 1, (0, 0, 0))
            game_display.blit(label, (10, 540))
            label = font.render("Distance: " + str(self.best_child.location.dist(self.target_location)), 1, (0, 0, 0))
            game_display.blit(label, (10, 570))

            pygame.display.update()

            clock.tick(FPS)

        self.print_stats()
        pygame.display.quit()
        pygame.quit()

    def print_stats(self):
        print("Valor de correctitud: ", self.best_child.fitness(self.target_location))
        print("Localización óptima: ", self.best_child.location)
        print("Distancia al objetivo: ", self.best_child.location.dist(self.target_location))

In [ ]:

genetic = Genetic("Smart Maze",700,600,100,40, 40, 100, 0.1, [(400, 0, 20, 300),(0, 200, 200, 300)])
genetic.simulate_with_graphics()


C:\Users\JoseSalas\Anaconda3\lib\site-packages\ipykernel_launcher.py:102: DeprecationWarning: This function is deprecated. Please call randint(1, 250 + 1) instead
